In [ ]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


## 충남 Cluster 148

In [3]:
chung_148 = pd.read_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/modeling data/chungnam_148.csv')
display(chung_148)
chung_148.isnull().sum()

,date,accident_num,평균기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 이슬점온도(°C),year,month,day,측정소명,O3,PM10,PM2.5,season,요일,workday
0,2020-01-01,2.0,0.3,0.0,0.7,68.1,-5.0,2020,1,1,대천2동,0,0.0,1.0,winter,수요일,1
1,2020-01-02,0.0,1.8,0.0,0.8,73.4,-2.5,2020,1,2,대천2동,0,1.0,2.0,winter,목요일,0
2,2020-01-03,0.0,2.5,0.0,0.6,73.8,-1.9,2020,1,3,대천2동,0,1.0,2.0,winter,금요일,0
3,2020-01-04,0.0,1.8,0.0,0.9,76.0,-2.1,2020,1,4,대천2동,0,1.0,2.0,winter,토요일,1
4,2020-01-05,5.0,2.0,0.0,1.3,54.9,-6.8,2020,1,5,대천2동,0,1.0,1.0,winter,일요일,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2023-01-27,0.0,-5.0,0.0,2.4,61.1,-11.7,2023,1,27,대천2동,0,0.0,0.0,winter,금요일,0
1123,2023-01-28,0.0,-5.6,0.0,0.9,73.9,-9.7,2023,1,28,대천2동,0,0.0,0.0,winter,토요일,1
1124,2023-01-29,0.0,0.0,0.0,1.8,67.0,-5.5,2023,1,29,대천2동,0,1.0,0.0,winter,일요일,1
1125,2023-01-30,0.0,-0.5,0.0,0.7,58.5,-8.2,2023,1,30,대천2동,0,1.0,0.0,winter,월요일,0


date            0
accident_num    0
평균기온(°C)        0
일강수량(mm)        0
평균 풍속(m/s)      1
평균 상대습도(%)      0
평균 이슬점온도(°C)    0
year            0
month           0
day             0
측정소명            0
O3              0
PM10            1
PM2.5           1
season          0
요일              0
workday         0
dtype: int64

In [4]:
chung_148 = chung_148.dropna(axis=0)
chung_148.isnull().sum()

date            0
accident_num    0
평균기온(°C)        0
일강수량(mm)        0
평균 풍속(m/s)      0
평균 상대습도(%)      0
평균 이슬점온도(°C)    0
year            0
month           0
day             0
측정소명            0
O3              0
PM10            0
PM2.5           0
season          0
요일              0
workday         0
dtype: int64

In [5]:
chung_148['date'] = pd.to_datetime(chung_148['date'])
chung_148['accident_hpn'] = chung_148['accident_num'].apply(lambda x: 1 if x > 0 else 0)
chung_148['accident_hpn'].value_counts()/chung_148['accident_hpn'].value_counts().sum()

0    0.516014
1    0.483986
Name: accident_hpn, dtype: float64

In [6]:
X = chung_148.drop(['date','accident_num','측정소명','accident_hpn'], axis = 1)
y = chung_148['accident_hpn']

In [7]:
print(X.shape)
print(y.shape)

(1124, 14)
(1124,)


In [8]:
X_trn, X_tes, y_trn, y_tes = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

print(X_trn.shape)
print(X_tes.shape)
print(y_trn.shape)
print(y_tes.shape)

(899, 14)
(225, 14)
(899,)
(225,)


In [9]:
cat_features = ['year', 'month','day','season','요일']
num_features = ['평균기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','평균 이슬점온도(°C)','O3','PM10','PM2.5']

enc = OneHotEncoder(handle_unknown = 'ignore')
scaler = MinMaxScaler()

enc.fit(X_trn[cat_features])
scaler.fit(X_trn[num_features])

train_oh = enc.transform(X_trn[cat_features]).toarray()
test_oh = enc.transform(X_tes[cat_features]).toarray()

train_num = scaler.transform(X_trn[num_features])
test_num = scaler.transform(X_tes[num_features])

print(train_oh.shape)
print(test_oh.shape)


(899, 58)
(225, 58)


In [10]:
X_trn = np.concatenate([train_num,train_oh,X_trn['workday'].values.reshape(-1,1)], axis = 1)
X_tes = np.concatenate([test_num,test_oh,X_tes['workday'].values.reshape(-1,1)], axis = 1)

print(X_trn.shape)
print(X_tes.shape)

(899, 67)
(225, 67)


In [11]:
class TrnDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len



In [12]:
class TesDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len



In [13]:
train_data = TrnDataset(X_trn, y_trn.values)
test_data = TesDataset(X_tes, y_tes.values)

train_loader = DataLoader(train_data,batch_size=100,shuffle=True)
test_loader = DataLoader(test_data,batch_size=100,shuffle=True)

In [14]:
arr, label = next(iter(train_loader))
print(arr.shape, label.shape)
arr2, label2 = next(iter(test_loader))
print(arr2.shape, label2.shape)

torch.Size([100, 67]) torch.Size([100])
torch.Size([100, 67]) torch.Size([100])


In [15]:
latent_size = 4

In [16]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    self.encoder = nn.Sequential(
                      nn.Linear(67,64),
                      nn.ReLU(),
                      nn.Linear(64,32),
                      nn.ReLU(),
                      nn.Linear(32,16),
                      nn.ReLU(),
                      nn.Linear(16,latent_size),
                      nn.ReLU())

  def forward(self,x):
    return self.encoder(x)


class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    self.decoder = nn.Sequential(
                      nn.Linear(latent_size, 16),
                      nn.ReLU(),
                      nn.Linear(16,32),
                      nn.ReLU(),
                      nn.Linear(32,64),
                      nn.ReLU(),
                      nn.Linear(64,67),
                      nn.Sigmoid())

  def forward(self,x):
    return self.decoder(x)

In [17]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr = 0.001)

In [18]:
for ep in range(500):

  running_loss = 0.0

  for images, label in train_loader:
    inputs = images.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    outputs = decoder(latent_var)
    loss = criterion(inputs, outputs)
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  cost = running_loss/len(train_loader)
  print('[%d]loss:%.3f'%(ep,cost))
  torch.save(encoder.state_dict(), './models/en_pth')
  torch.save(decoder.state_dict(), './models/de_pth')

[0]loss:0.227
[1]loss:0.215
[2]loss:0.195
[3]loss:0.162
[4]loss:0.110
[5]loss:0.079
[6]loss:0.076
[7]loss:0.073
[8]loss:0.072
[9]loss:0.071
[10]loss:0.071
[11]loss:0.070
[12]loss:0.070
[13]loss:0.070
[14]loss:0.070
[15]loss:0.070
[16]loss:0.070
[17]loss:0.070
[18]loss:0.070
[19]loss:0.070
[20]loss:0.070
[21]loss:0.070
[22]loss:0.070
[23]loss:0.070
[24]loss:0.070
[25]loss:0.070
[26]loss:0.070
[27]loss:0.070
[28]loss:0.070
[29]loss:0.070
[30]loss:0.070
[31]loss:0.069
[32]loss:0.069
[33]loss:0.069
[34]loss:0.068
[35]loss:0.067
[36]loss:0.066
[37]loss:0.066
[38]loss:0.065
[39]loss:0.064
[40]loss:0.063
[41]loss:0.063
[42]loss:0.063
[43]loss:0.062
[44]loss:0.062
[45]loss:0.062
[46]loss:0.061
[47]loss:0.061
[48]loss:0.061
[49]loss:0.061
[50]loss:0.061
[51]loss:0.060
[52]loss:0.060
[53]loss:0.060
[54]loss:0.060
[55]loss:0.060
[56]loss:0.060
[57]loss:0.060
[58]loss:0.059
[59]loss:0.059
[60]loss:0.059
[61]loss:0.059
[62]loss:0.059
[63]loss:0.059
[64]loss:0.058
[65]loss:0.058
[66]loss:0.058
[67]l

In [19]:
#forward training data
encoder.load_state_dict(torch.load('/content/models/en_pth'))
decoder.load_state_dict(torch.load('/content/models/de_pth'))

#latent vector
latent_features_trn = []
actual_trn = []

with torch.no_grad():
  for images, labels in train_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)

    latent_features_trn+=latent_var.cpu().tolist()
    actual_trn+=labels.cpu().tolist()

In [20]:
#forward test data
encoder.load_state_dict(torch.load('/content/models/en_pth'))
decoder.load_state_dict(torch.load('/content/models/de_pth'))

#latent vector
latent_features_tes = []
actual_tes = []

with torch.no_grad():
  for images, labels in test_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)

    latent_features_tes+=latent_var.cpu().tolist()
    actual_tes+=labels.cpu().tolist()

In [21]:
latent_features_trn = np.array(latent_features_trn)
latent_features_tes = np.array(latent_features_tes)
actual_tes = np.array(actual_tes)
actual_trn = np.array(actual_trn)

In [22]:
latent_features_trn=pd.DataFrame(latent_features_trn)
latent_features_tes=pd.DataFrame(latent_features_tes)
actual_tes = pd.DataFrame(actual_tes)
actual_trn = pd.DataFrame(actual_trn)

In [23]:
latent_features_trn.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 148/latent_features_trn.csv')
latent_features_tes.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 148/latent_features_tes.csv')
actual_tes.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 148/actual_tes.csv')
actual_trn.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 148/actual_trn.csv')

In [24]:
actual_trn = actual_trn.values.reshape(-1,)
actual_tes = actual_tes.values.reshape(-1,)

In [37]:
clf = RandomForestClassifier(n_estimators = 400, criterion = 'entropy',min_samples_split = 10, random_state = 0)
clf.fit(latent_features_trn, actual_trn)
y_trn_pred = clf.predict(latent_features_trn)

print(classification_report(actual_trn,y_trn_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92       464
           1       0.92      0.91      0.91       435

    accuracy                           0.92       899
   macro avg       0.92      0.92      0.92       899
weighted avg       0.92      0.92      0.92       899



In [38]:
y_pred = clf.predict(latent_features_tes)
print(classification_report(actual_tes,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.67      0.66       116
           1       0.63      0.61      0.62       109

    accuracy                           0.64       225
   macro avg       0.64      0.64      0.64       225
weighted avg       0.64      0.64      0.64       225

